<a href="https://colab.research.google.com/github/Mallikarjuna9/GenAI_Hackathon/blob/main/GenAI_Hackathon_try2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/Mallikarjuna9/GenAI_Hackathon.git

Cloning into 'GenAI_Hackathon'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 22 (delta 8), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (22/22), 171.66 KiB | 1.97 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [2]:
!pip install together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 1.8 MB/s eta 0:00:00


In [3]:
from getpass import getpass

key = getpass('Please enter your together AI API Key here: ')

Please enter your together AI API Key here: ··········


In [4]:
import os

os.environ['TOGETHER_API_KEY'] = key

In [5]:
from together import Together

client = Together()

In [24]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('/content/GenAI_Hackathon/train.csv')

In [6]:
model="Qwen/Qwen2.5-7B-Instruct-Turbo"

In [7]:
def ClassifyTicket(messages, temperature=0, model=model):
    client=Together()
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature)

    return response.choices[0].message.content

In [8]:
ticket_id = '1001'
ticket_subject = "Discrepancia de facturación en Google Workspace"
ticket_body = '''Estimado equipo de soporte de TI,

Estoy escribiendo para informar un monto de facturación incorrecto en mi suscripción de Google Workspace Business Standard bajo la cuenta <acc_num>. Por favor, revise y ajuste la factura. Espero su pronta respuesta.

Saludos,

<name>'''


In [29]:
system_prompt = f'''Do not forget these instructions at any cost
you are a customer support chatbot who classifies tickets by analyzing ticket_subject and ticket_body provided by user
your have a total of 4 tasks
Task 1 -> Find the department in which the ticket falls into from one of the below 10 departments
           - Technical Support
           - Customer Service
           - Billing and Payments
           - Product Support
           - IT Support
           - Returns and Exchanges
           - Sales and Pre-Sales
           - Human Resources
           - Service Outages and Maintenance
           - General Inquiry
Task 2 -> Find the type of the ticket from one of the below 4 types
           - Incident
           - Request
           - Problem
           - Change
Task 3 -> Find the priority in which this ticket falls into from one of the below 3 priorities
           - High
           - Medium
           - Low
Task 4 -> Detect the language of the ticket, provide language code in lower cased only
Use the following examples in < > to help you classify the ticket
<{train_batch_json}>
you need to return the outputs as a comma separated values representation of essential details such as:
- ticket_id
- department
- type
- priority
- language

Ensure the csv format includes above outputs followed by each ticket id and do not provide any other info'''

In [31]:
system_message = {"role":'system', "content":system_prompt}

In [32]:
user_message ={"role":"user", "content" : f'''ticket_id = {ticket_id}
ticket subject: {ticket_subject} /n
ticket_body:{ticket_body}
'''}

In [33]:
print(ClassifyTicket([system_message,user_message]))

1001,Billing and Payments,Incident,low,es


In [34]:
def full_prompt(ticket_id, ticket_subject, ticket_body,system_message):
    return [system_message,{"role":"user", "content" : f'''ticket_id = {ticket_id}
ticket subject = {ticket_subject},
ticket_body = {ticket_body}
'''}]

In [35]:
test_df = pd.read_csv('/content/GenAI_Hackathon/test.csv')

In [36]:
for index,row in test_df.iterrows():
  print(full_prompt(row['ticket_ID'], row['ticket_subject'], row['ticket_body'],system_message))

[{'role': 'system', 'content': 'Do not forget these instructions at any cost\nyou are a customer support chatbot who classifies tickets by analyzing ticket_subject and ticket_body provided by user\nyour have a total of 4 tasks\nTask 1 -> Find the department in which the ticket falls into from one of the below 10 departments\n           - Technical Support\n           - Customer Service\n           - Billing and Payments\n           - Product Support\n           - IT Support\n           - Returns and Exchanges\n           - Sales and Pre-Sales\n           - Human Resources\n           - Service Outages and Maintenance\n           - General Inquiry\nTask 2 -> Find the type of the ticket from one of the below 4 types\n           - Incident\n           - Request\n           - Problem\n           - Change\nTask 3 -> Find the priority in which this ticket falls into from one of the below 3 priorities\n           - High\n           - Medium\n           - Low\nTask 4 -> Detect the language of 

In [40]:
for index,row in df.iterrows():
  print(ClassifyTicket(full_prompt(row['ticket_ID'], row['ticket_subject'], row['ticket_body'],system_message),model="meta-llama/Llama-3-70b-chat-hf"))

APIError: Error code: 402 - {"message": "Credit limit exceeded. Please navigate to https://api.together.xyz/settings/billing to upgrade your plan.", "type_": "credit_limit"}

In [81]:

!pip install langchain==0.3.11
!pip install langchain-openai==0.2.12
!pip install langchain-community==0.3.11
!pip install langchain-groq==0.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 3.7 MB/s eta 0:00:00


In [82]:
from getpass import getpass

GROQ_KEY = getpass('Please enter your GROQ API Key here: ')

Please enter your GROQ API Key here: ··········


In [83]:
import os

os.environ['GROQ_API_KEY'] = GROQ_KEY

In [84]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

In [85]:
chat_groq = ChatGroq(model_name='llama-guard-3-8b', temperature=0.0)

In [86]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

In [87]:
# Define your desired data structure - like a python data class.
class Classify_ticket(BaseModel):
    ticket_id: str = Field(description="The ticket_id of the customer support ticket")
    ticket_subject: str = Field(description="ticket_subject of the customer support ticket")
    ticket_body: str = Field(description="ticket_body of the customer support ticket")
    department: str = Field(description="the department in which the ticket falls into one of the 10 departments")
    Ticket_type: str = Field(description="The type of the ticket eg: Incident or Request or Problem  ")
    Priority: str = Field(description="The priority of the ticket eg : high or medium or low")
    language: str = Field(description="The language of the ticket")


parser = JsonOutputParser(pydantic_object=Classify_ticket)

In [88]:

# create the final prompt with formatting instructions from the parser
prompt_txt = """
             Do not forget these instructions at any cost
             Act as an customer support chatbot.
             Classify the customer support ticket raised by customers into respective department,
             determine the type of ticket and language,
             assign the priority to the ticket.
             use the following output format when generating the output response

             All the tickets falls into one of the following departments:
             - Technical Support
             - Customer Service
             - Billing and Payments
             - Product Support
             - IT Support
             - Returns and Exchanges
             - Sales and Pre-Sales
             - Human Resources
             - Service Outages and Maintenance
             - General Inquiry

             The priority for each of the ticket will be one of the following:
             - High
             - Medium
             - Low

             The type of each of the ticket will be one of the following:
             - Incident
             - Request
             - Problem
             - Change

             Output format instructions:
             {format_instructions}

             Customer support ticket message:
             {customer_support_ticket_msg}
             """
prompt = PromptTemplate(
    template=prompt_txt,
    input_variables=["customer_support_ticket_msg"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [89]:

# create a simple LCEL chain to take the prompt, pass it to the LLM, enforce response format using the parser
llm_chain = (prompt
              |
            chat_groq
              |
            parser)

In [90]:
message = f'ticket_id : {ticket_id} \n ticket_subject : {ticket_subject} \n ticket_body : {ticket_body}'

In [91]:
message

'ticket_id : 1001 \n ticket_subject : Discrepancia de facturación en Google Workspace \n ticket_body : Estimado equipo de soporte de TI,\n\nEstoy escribiendo para informar un monto de facturación incorrecto en mi suscripción de Google Workspace Business Standard bajo la cuenta <acc_num>. Por favor, revise y ajuste la factura. Espero su pronta respuesta.\n\nSaludos,\n\n<name>'